# Exemple de notebook - premières analyses des données SISE-Eaux


In [1]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données.

import duckdb
import pandas as pd

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [2]:
# Affichons les tables

con.table("sise_communes").show()

┌──────────────┬────────────────────────────┬──────────────────────┬───────────┬─────────────────────────────────┬────────────┐
│ inseecommune │         nomcommune         │       quartier       │ cdreseau  │            nomreseau            │ debutalim  │
│   varchar    │          varchar           │       varchar        │  varchar  │             varchar             │    date    │
├──────────────┼────────────────────────────┼──────────────────────┼───────────┼─────────────────────────────────┼────────────┤
│ 01001        │ ABERGEMENT-CLEMENCIAT (L') │ -                    │ 001000556 │ BDS ST DIDIER/CHALARONNE        │ 2010-09-07 │
│ 01002        │ ABERGEMENT-DE-VAREY (L')   │ -                    │ 001000369 │ L'ABERGEMENT-DE-VAREY           │ 2010-09-07 │
│ 01004        │ AMBERIEU-EN-BUGEY          │ Vareilles            │ 001000248 │ AMBERIEU VAREILLES              │ 2010-09-07 │
│ 01004        │ AMBERIEU-EN-BUGEY          │ St Germain_Brédevent │ 001000249 │ AMBERIEU SAINT GERMAIN 

In [3]:
con.table("sise_prelevements").show()

┌─────────┬───────────┬───────────────────┬─────────────────┬───────────────┬─────────────────────────────────────┬───────────────┬───────────────┬────────────┬───────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────┬───────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┐
│ cddept  │ cdreseau  │ inseecommuneprinc │ nomcommuneprinc │ cdreseauamont │           nomreseauamont            │ pourcentdebit │ referenceprel │  dateprel  │ heureprel │                                                                                 conclusionprel                                                                                  │              ugelib               │               distrlib                │ 

In [4]:
con.table("sise_resultats").show()

┌─────────┬───────────────┬─────────────────────┬─────────────┬─────────────────────────────┬─────────────────────────────┬─────────────────┬─────────────┬───────────┬──────────────────────────┬──────────────────────────┬──────────────────┬────────────┬────────────┬─────────────┬─────────────┬──────────────┐
│ cddept  │ referenceprel │ cdparametresiseeaux │ cdparametre │       libmajparametre       │       libminparametre       │ libwebparametre │ qualitparam │ insituana │          rqana           │ cdunitereferencesiseeaux │ cdunitereference │ limitequal │  refqual   │ valtraduite │  casparam   │ referenceanl │
│ varchar │    varchar    │       varchar       │    int64    │           varchar           │           varchar           │     varchar     │   varchar   │  varchar  │         varchar          │         varchar          │     varchar      │  varchar   │  varchar   │   double    │   varchar   │   varchar    │
├─────────┼───────────────┼─────────────────────┼─────────────┼───────

In [5]:
# Chargeons la table sise_communes dans un pandas dataframe, et calculons le nombre de communes

communes = con.table("sise_communes").to_df()
nombre_de_communes = communes.nunique()["inseecommune"]
print(f"nombre_de_communes = {nombre_de_communes}")


nombre_de_communes = 34880


In [9]:
# Faisons une requête SQL en utilisant duckdb via la librarie python pour lister les substances qui ont été recherchées
# et les trier par ordre décroissant de leur nombre d'occurrences

con.sql("""
    SELECT libmajparametre, COUNT(*) as count
    FROM sise_resultats
    GROUP BY libmajparametre
    ORDER BY count DESC
""").show()


┌─────────────────────────────────────┬────────┐
│           libmajparametre           │ count  │
│               varchar               │ int64  │
├─────────────────────────────────────┼────────┤
│ PH                                  │ 338786 │
│ CONDUCTIVITÉ À 25°C                 │ 265188 │
│ TEMPÉRATURE DE L'EAU                │ 258858 │
│ CHLORE LIBRE                        │ 257631 │
│ ESCHERICHIA COLI /100ML - MF        │ 255609 │
│ ENTÉROCOQUES /100ML-MS              │ 255593 │
│ BACTÉRIES COLIFORMES /100ML-MS      │ 255539 │
│ TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU       │ 255307 │
│ CHLORE TOTAL                        │ 254918 │
│ BACT. AÉR. REVIVIFIABLES À 36°-44H  │ 254690 │
│          ·                          │      · │
│          ·                          │      · │
│          ·                          │      · │
│ BENZO(J)FLUORANTHÈNE                │      1 │
│ GIARDIA SPP                         │      1 │
│ CHLOROTHALONIL MÉTABOLITE R611965   │      1 │
│ RÉSIDU SEC À 180° 

In [7]:
# Maintenant configurons l'extension SQL pour Jupyter...

%load_ext sql
%sql duckdb:///../../database/data.duckdb

# ...et faisons la même requête SQL en utilisant l'extension SQL pour Jupyter

%sql SELECT libmajparametre, COUNT(*) as count FROM sise_resultats GROUP BY libmajparametre ORDER BY count DESC;

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

Running query in 'duckdb:///../../database/data.duckdb'

libmajparametre,count
PH,338786
CONDUCTIVITÉ À 25°C,265188
TEMPÉRATURE DE L'EAU,258858
CHLORE LIBRE,257631
ESCHERICHIA COLI /100ML - MF,255609
ENTÉROCOQUES /100ML-MS,255593
BACTÉRIES COLIFORMES /100ML-MS,255539
TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,255307
CHLORE TOTAL,254918
BACT. AÉR. REVIVIFIABLES À 36°-44H,254690


In [8]:
# Enfin, terminons par lister les prélèvements effectués dans une commune donnée

nomcommune = "TOULOUSE"

con.sql(f"""
    SELECT *
    FROM sise_prelevements
    WHERE nomcommuneprinc = '{nomcommune}'
""").show()

┌─────────┬───────────┬───────────────────┬─────────────────┬───────────────┬───────────────────────────┬───────────────┬───────────────┬────────────┬───────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┬──────────┬────────────────────┬───────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┐
│ cddept  │ cdreseau  │ inseecommuneprinc │ nomcommuneprinc │ cdreseauamont │      nomreseauamont       │ pourcentdebit │ referenceprel │  dateprel  │ heureprel │                                             conclusionprel                                              │       ugelib       │ distrlib │       moalib       │ plvconformitebacterio │ plvconformitechimique │ plvconformitereferencebact │ plvconformitereferencechim │
│ varchar │  varchar  │      varchar      │     varchar     │    varchar    │          varchar          │    varchar    │    varchar    │    dat